# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with `mongoimport --type json -d uk_food -c establishments --drop --jsonArray establishments.json'

In [1]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [2]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [3]:
# confirm that our new database was created
mongo.list_database_names()


['admin', 'config', 'local', 'met', 'petsitly_marketing', 'uk_food']

In [4]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [5]:
# review the collections in our new database
print(db.list_collection_names())

['establishments']


In [6]:
# review a document in the establishments collection
document = establishments.find_one()
document

NameError: name 'establishments' is not defined

In [ ]:
# assign the collection to a variable
establishments = db['establishments']

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [ ]:
# Create a dictionary for the new restaurant data
penang = {
    "BusinessName":"Penang Flavours",
    "BusinessType":"Restaurant/Cafe/Canteen",
    "BusinessTypeID":"",
    "AddressLine1":"Penang Flavours",
    "AddressLine2":"146A Plumstead Rd",
    "AddressLine3":"London",
    "AddressLine4":"",
    "PostCode":"SE18 7DY",
    "Phone":"",
    "LocalAuthorityCode":"511",
    "LocalAuthorityName":"Greenwich",
    "LocalAuthorityWebSite":"http://www.royalgreenwich.gov.uk",
    "LocalAuthorityEmailAddress":"health@royalgreenwich.gov.uk",
    "scores":{
        "Hygiene":"",
        "Structural":"",
        "ConfidenceInManagement":""
    },
    "SchemeType":"FHRS",
    "geocode":{
        "longitude":"0.08384000",
        "latitude":"51.49014200"
    },
    "RightToReply":"",
    "Distance":4623.9723280747176,
    "NewRatingPending":True
}

In [ ]:
# Insert the new restaurant into the collection
establishments.insert_one(penang)

In [ ]:
# Check that the new restaurant was inserted
penang_find = establishments.find_one({'BusinessName':'Penang Flavours'})
pprint(penang_find)

2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [ ]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
query = {'BusinessType':'Restaurant/Cafe/Canteen'}
fields = {'BusinessTypeID':1, 'BusinessType':1, '_id':0}
results = establishments.find(query, fields)
for result in results:
    pprint(result)

3. Update the new restaurant with the `BusinessTypeID` you found.

In [ ]:
# Update the new restaurant with the correct BusinessTypeID
query = {'BusinessName':'Penang Flavours'}
update = {'$set':{'BusinessTypeID':1}}
results = establishments.update_one(query,update)

In [ ]:
# Confirm that the new restaurant was updated
query = {'BusinessName':'Penang Flavours'}
penang_docu = establishments.find_one(query)
penang_docu

4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [ ]:
# Find how many documents have LocalAuthorityName as "Dover"
query = {'LocalAuthorityName': 'Dover'}
count = establishments.count_documents(query)
count

In [ ]:
# Delete all documents where LocalAuthorityName is "Dover"
query = {'LocalAuthorityName': 'Dover'}
deleted = establishments.delete_many(query)

In [ ]:
# Check if any remaining documents include Dover
query = {'LocalAuthorityName': 'Dover'}
no_dover = establishments.find_one(query)
if no_dover =='':
    print('No, Dover IS present')
else:
    print('Yes, Dover NOT present')

In [ ]:
# Check that other documents remain with 'find_one'
check = establishments.find_one()
pprint(check)

5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [ ]:
# Change the data type from String to Decimal for longitude and latitude
query = [{'$set': {
            'geocode.latitude': {
                '$convert': {
                    'input': '$geocode.latitude',
                    'to': 'decimal',
                    'onError': 0.0}},
            'geocode.longitude': {
                '$convert': {
                    'input': '$geocode.longitude',
                    'to': 'decimal',
                    'onError': 0.0}}}}]

establishments.update_many({},query)

Use `update_many` to convert `RatingValue` to integer numbers.

In [ ]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

In [ ]:
# Change the data type from String to Integer for RatingValue
establishments.update_many({}, [{'$set': {'RatingValue': {'$toInt': '$RatingValue'}}}])

In [ ]:
# Check that the coordinates and rating value are now numbers

# If either of these come back with nothing, then I know the types have changed
geocode = {
    '$or': [
        {'geocode.latitude': {'$type': 'string'}},
        {'geocode.longitude': {'$type': 'string'}}
    ]
}
rating = {
    'RatingValue': {'$type': 'string'}
}
geocode_str = establishments.count_documents(geocode)
rating_str = establishments.count_documents(rating)

print(geocode_str)
print(rating_str)